
# Trader Behavior vs Market Sentiment — End‑to‑End Analysis (Colab)

**Objective:** Explore the relationship between trader performance and market sentiment (Fear/Greed) and derive actionable insights for trading strategy.


In [ ]:

# === Setup: installs & imports (run once) ===
!pip -q install gdown pandas numpy matplotlib scipy scikit-learn plotly

import os, math, json, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

warnings.filterwarnings("ignore")

# Folders (mirror required structure)
ROOT = os.getcwd()
CSV_DIR = os.path.join(ROOT, "csv_files")
OUT_DIR = os.path.join(ROOT, "outputs")
os.makedirs(CSV_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

print("Working directory:", ROOT)
print("csv_files ->", CSV_DIR)
print("outputs   ->", OUT_DIR)



## 1) Data Access (Google Drive IDs)
If the links are public, use `gdown` with the file IDs. Otherwise, mount Drive and copy files manually.


In [ ]:

# Update these IDs if the company shares new links
HYPERLIQUID_FILE_ID = "1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs"
FEARGREED_FILE_ID   = "1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf"

# Choose preferred method: (A) direct gdown or (B) manual upload/mount
USE_GDOWN = True

hyperliquid_path = os.path.join(CSV_DIR, "hyperliquid_raw.csv")
feargreed_path   = os.path.join(CSV_DIR, "fear_greed.csv")

if USE_GDOWN:
    import gdown
    # Force Google Drive download by ID and save to known filenames
    gdown.download(id=HYPERLIQUID_FILE_ID, output=hyperliquid_path, quiet=False)
    gdown.download(id=FEARGREED_FILE_ID, output=feargreed_path, quiet=False)
else:
    print("Skipping gdown. Upload CSVs to csv_files/ named as shown above.")

print("Files expected:")
print(" -", hyperliquid_path, os.path.exists(hyperliquid_path))
print(" -", feargreed_path, os.path.exists(feargreed_path))



## 2) Robust Loaders & Cleaning
Handles CSV/JSON/NDJSON; parses time and normalizes fields.


In [ ]:

def load_table(path):
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    name = os.path.basename(path).lower()
    try:
        if name.endswith(".csv"):
            return pd.read_csv(path)
        elif name.endswith(".json"):
            return pd.read_json(path)
        elif name.endswith(".ndjson") or name.endswith(".jsonl"):
            return pd.read_json(path, lines=True)
        else:
            # Attempt CSV by default
            return pd.read_csv(path)
    except Exception as e:
        print("Primary load failed, trying different encodings...")
        for enc in ["utf-8", "latin1"]:
            try:
                return pd.read_csv(path, encoding=enc)
            except Exception:
                pass
        raise e

trades_raw = load_table(hyperliquid_path)
sent_raw   = load_table(feargreed_path)

print("Trades shape:", trades_raw.shape)
print("Sentiment shape:", sent_raw.shape)
print("Trades columns:", list(trades_raw.columns)[:20])
print("Sent columns:", list(sent_raw.columns))

# --- Standardize column names ---
trades = trades_raw.rename(columns={
    "execution price": "execution_price",
    "start position": "start_position",
    "closedPnL": "closedpnl",
    "symbol": "symbol",
    "account": "account",
    "size": "size",
    "side": "side",
    "time": "time",
    "event": "event",
    "leverage": "leverage"
})
sent = sent_raw.rename(columns={
    "Date": "date",
    "Classification": "classification",
    "classification": "classification"
})

# --- Parse time/date ---
# Trades time may be ms/seconds epoch or ISO string
def to_datetime_safe(x):
    try:
        # numeric epoch?
        if pd.api.types.is_number(x):
            # Heuristic: treat > 10^12 as ms
            if x > 1e12:
                return pd.to_datetime(x, unit="ms", utc=True)
            else:
                return pd.to_datetime(x, unit="s", utc=True)
        return pd.to_datetime(x, utc=True, errors="coerce")
    except Exception:
        return pd.NaT

if "time" in trades.columns:
    trades["time_dt"] = trades["time"].apply(to_datetime_safe)
else:
    # Try other names
    for cand in ["timestamp", "created_at", "executed_at"]:
        if cand in trades.columns:
            trades["time_dt"] = trades[cand].apply(to_datetime_safe)
            break

trades["date"] = trades["time_dt"].dt.tz_convert("UTC").dt.date

# Normalize types
for col in ["execution_price", "size", "leverage", "closedpnl", "start_position"]:
    if col in trades.columns:
        trades[col] = pd.to_numeric(trades[col], errors="coerce")

# Clean 'side'
if "side" in trades.columns:
    trades["side"] = trades["side"].astype(str).str.strip().str.lower()
    trades["side"] = trades["side"].replace({"buy":"long","sell":"short"})
    trades["side"] = trades["side"].where(trades["side"].isin(["long","short"]), other="unknown")

# Sentiment date parse
if "date" in sent.columns:
    sent["date"] = pd.to_datetime(sent["date"], errors="coerce").dt.date

# Sentiment label normalize
if "classification" in sent.columns:
    sent["classification"] = sent["classification"].astype(str).str.strip().str.title()
    # Map to two classes where possible
    sent["sent_bin"] = sent["classification"].map({
        "Fear": "Fear",
        "Greed": "Greed"
    }).fillna(sent["classification"])  # keep original if more granular
else:
    sent["sent_bin"] = "Unknown"

# Merge
merged = pd.merge(trades, sent[["date","sent_bin","classification"]].drop_duplicates(),
                  on="date", how="left")

print("Merged shape:", merged.shape)
merged.to_csv(os.path.join(CSV_DIR, "trades_merged.csv"), index=False)
print("Saved:", os.path.join(CSV_DIR, "trades_merged.csv"))



## 3) Feature Engineering
Create per-trade metrics, leverage bands, win flag, ROI proxy.


In [ ]:

df = merged.copy()

# Win flag
if "closedpnl" in df.columns:
    df["win"] = (df["closedpnl"] > 0).astype(int)
else:
    df["win"] = np.nan

# Leverage bands
if "leverage" in df.columns:
    df["lev_band"] = pd.cut(df["leverage"], bins=[-np.inf, 2, 5, 10, 20, 50, np.inf],
                            labels=["<=2x","2–5x","5–10x","10–20x","20–50x",">50x"])

# ROI proxy (approx): closedPnL / (abs(execution_price * size) / max(leverage,1))
def roi_proxy(row):
    ep = row.get("execution_price", np.nan)
    sz = row.get("size", np.nan)
    lev = row.get("leverage", 1.0)
    pnl = row.get("closedpnl", np.nan)
    if pd.isna(ep) or pd.isna(sz) or pd.isna(pnl):
        return np.nan
    denom = abs(ep * sz) / max(lev, 1.0)
    return pnl / denom if denom not in [0, np.inf, -np.inf] else np.nan

df["roi_proxy"] = df.apply(roi_proxy, axis=1)

# Save engineered
df.to_csv(os.path.join(CSV_DIR, "trades_engineered.csv"), index=False)
print("Saved:", os.path.join(CSV_DIR, "trades_engineered.csv"))



## 4) EDA & Visuals
All charts will be saved to `outputs/`.


In [ ]:

def save_bar(series, title, fname):
    plt.figure()
    series.plot(kind="bar")
    plt.title(title)
    plt.tight_layout()
    path = os.path.join(OUT_DIR, fname)
    plt.savefig(path, dpi=150)
    plt.close()
    print("Saved chart:", path)

def save_boxplot(data, by_col, value_col, title, fname):
    plt.figure()
    data.boxplot(column=value_col, by=by_col)
    plt.suptitle("")
    plt.title(title)
    plt.tight_layout()
    path = os.path.join(OUT_DIR, fname)
    plt.savefig(path, dpi=150)
    plt.close()
    print("Saved chart:", path)

# Trade counts by sentiment
if "sent_bin" in df.columns:
    counts = df["sent_bin"].value_counts().sort_index()
    counts.to_csv(os.path.join(CSV_DIR, "counts_by_sentiment.csv"))
    save_bar(counts, "Trade Counts by Sentiment", "counts_by_sentiment.png")

# Mean closedPnL by sentiment
if "closedpnl" in df.columns and "sent_bin" in df.columns:
    mean_pnl = df.groupby("sent_bin")["closedpnl"].mean().sort_index()
    mean_pnl.to_csv(os.path.join(CSV_DIR, "mean_pnl_by_sentiment.csv"))
    save_bar(mean_pnl, "Mean closedPnL by Sentiment", "mean_pnl_by_sentiment.png")
    # Boxplot
    save_boxplot(df.dropna(subset=["closedpnl","sent_bin"]), "sent_bin", "closedpnl",
                 "closedPnL Distribution by Sentiment", "box_closedpnl_by_sentiment.png")

# Win rate by sentiment × side × leverage band
for group_cols, tag in [ (["sent_bin"], "sent"),
                         (["sent_bin","side"], "sent_side"),
                         (["sent_bin","lev_band"], "sent_lev") ]:
    if all(col in df.columns for col in group_cols):
        agg = df.groupby(group_cols)["win"].mean().reset_index().rename(columns={"win":"win_rate"})
        agg.to_csv(os.path.join(CSV_DIR, f"winrate_by_{tag}.csv"), index=False)

# Symbol × Sentiment mean ROI
if all(col in df.columns for col in ["symbol","sent_bin","roi_proxy"]):
    roi_pivot = df.pivot_table(index="symbol", columns="sent_bin", values="roi_proxy", aggfunc="mean")
    roi_pivot.to_csv(os.path.join(CSV_DIR, "roi_by_symbol_sentiment.csv"))



## 5) Statistical Tests
Test differences between Fear vs Greed for PnL and win rate.


In [ ]:

stats_out = {}

if "sent_bin" in df.columns:
    fear = df.loc[df["sent_bin"].str.contains("Fear", case=False, na=False)]
    greed = df.loc[df["sent_bin"].str.contains("Greed", case=False, na=False)]

    # PnL comparison
    if "closedpnl" in df.columns:
        a = fear["closedpnl"].dropna()
        b = greed["closedpnl"].dropna()
        if len(a) > 5 and len(b) > 5:
            # Non-parametric U-test
            u_stat, p_val = stats.mannwhitneyu(a, b, alternative="two-sided")
            stats_out["p_closedpnl_mwu"] = float(p_val)
            stats_out["closedpnl_mean_fear"] = float(a.mean())
            stats_out["closedpnl_mean_greed"] = float(b.mean())

    # Win-rate comparison via chi-square
    if "win" in df.columns:
        win_fear = fear["win"].dropna().astype(int)
        win_greed = greed["win"].dropna().astype(int)
        if len(win_fear) > 5 and len(win_greed) > 5:
            table = pd.crosstab(pd.concat([pd.Series(["Fear"]*len(win_fear)),
                                           pd.Series(["Greed"]*len(win_greed))], ignore_index=True),
                                pd.concat([win_fear, win_greed], ignore_index=True))
            chi2, p, dof, exp = stats.chi2_contingency(table)
            stats_out["p_winrate_chi2"] = float(p)
            stats_out["winrate_fear"] = float(win_fear.mean())
            stats_out["winrate_greed"] = float(win_greed.mean())

pd.DataFrame([stats_out]).to_csv(os.path.join(CSV_DIR, "stat_tests_summary.csv"), index=False)
print("Saved stats summary:", os.path.join(CSV_DIR, "stat_tests_summary.csv"))
stats_out



## 6) Predictive Modeling (Logistic Regression)
Predict `win` using sentiment, side, leverage, symbol, and size. This is a baseline.


In [ ]:

model_metrics = {}
work = df.dropna(subset=["win"]).copy()

# Feature set
cat_cols = [c for c in ["sent_bin","side","symbol"] if c in work.columns]
num_cols = [c for c in ["leverage","size","roi_proxy","execution_price"] if c in work.columns]

# One-hot encode categoricals
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
X_cat = enc.fit_transform(work[cat_cols]) if cat_cols else np.empty((len(work),0))
X_num = work[num_cols].fillna(0).to_numpy() if num_cols else np.empty((len(work),0))
X = np.hstack([X_cat, X_num])
y = work["win"].astype(int).to_numpy()

# Train/test split
if len(work) > 100:
    test_size = 0.2
else:
    test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

# Model
clf = LogisticRegression(max_iter=200, n_jobs=None)
clf.fit(X_train, y_train)
proba = clf.predict_proba(X_test)[:,1]
preds = (proba >= 0.5).astype(int)

# Metrics
auc = roc_auc_score(y_test, proba)
report = classification_report(y_test, preds, output_dict=True)
cm = confusion_matrix(y_test, preds).tolist()

model_metrics = {
    "auc": float(auc),
    "accuracy": float(report["accuracy"]),
    "precision_1": float(report.get("1", {}).get("precision", np.nan)),
    "recall_1": float(report.get("1", {}).get("recall", np.nan)),
    "cm": cm
}

pd.DataFrame([model_metrics]).to_csv(os.path.join(CSV_DIR, "model_metrics.csv"), index=False)
print("Saved:", os.path.join(CSV_DIR, "model_metrics.csv"))
model_metrics



## 7) Auto‑Summary Draft
Creates a draft insights file from current results. You can refine wording in the PDF report.


In [ ]:

lines = []
def fmt(x): 
    try:
        return f"{x:.4f}"
    except Exception:
        return str(x)

# Pull aggregates if they exist
try:
    mean_pnl = pd.read_csv(os.path.join(CSV_DIR, "mean_pnl_by_sentiment.csv"), index_col=0).iloc[:,0].to_dict()
except Exception:
    mean_pnl = {}

try:
    stats_summary = pd.read_csv(os.path.join(CSV_DIR, "stat_tests_summary.csv")).iloc[0].to_dict()
except Exception:
    stats_summary = {}

try:
    model_summary = pd.read_csv(os.path.join(CSV_DIR, "model_metrics.csv")).iloc[0].to_dict()
except Exception:
    model_summary = {}

lines.append("Key Findings:")
if mean_pnl:
    lines.append(f"- Mean closedPnL: { {k: fmt(v) for k,v in mean_pnl.items()} }")
if stats_summary:
    if "p_closedpnl_mwu" in stats_summary:
        lines.append(f"- PnL difference Fear vs Greed p-value (Mann-Whitney U): {fmt(stats_summary['p_closedpnl_mwu'])}")
    if "p_winrate_chi2" in stats_summary:
        lines.append(f"- Win-rate difference Fear vs Greed p-value (Chi-square): {fmt(stats_summary['p_winrate_chi2'])}")
    if "winrate_fear" in stats_summary and "winrate_greed" in stats_summary:
        lines.append(f"- Win-rate Fear={fmt(stats_summary['winrate_fear'])}, Greed={fmt(stats_summary['winrate_greed'])}")
if model_summary:
    lines.append(f"- Logistic baseline AUC={fmt(model_summary.get('auc'))}, Accuracy={fmt(model_summary.get('accuracy'))}")

lines.append("Recommendations:")
lines.append("- Consider reducing leverage during Fear days, especially if loss odds significantly increase.")
lines.append("- Focus on symbols and sides that historically perform better under each sentiment regime.")
lines.append("- Use sentiment-aware position sizing and tighter risk controls on adverse regimes.")

insight_path = os.path.join(OUT_DIR, "insights_draft.txt")
with open(insight_path, "w") as f:
    f.write("\n".join(lines))

print("Saved:", insight_path)



## 8) Create the Final PDF
- Open `ds_report_template.md`, paste in the insights and insert saved charts from `outputs/`.
- In Colab: **File → Print → Save as PDF** (or copy to Google Docs → File → Download → PDF).
- Name the final file `ds_report.pdf` and place it in the project root.
